# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [58]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import pprint

# Import API key
from api_keys_copy2 import geoapify_key

In [61]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bathsheba,13.2167,-59.5167,83.01,74,20,17.27,BB,1712878124
1,1,mvurwi,-17.0333,30.8500,59.34,97,100,7.18,ZW,1712878124
2,2,oroqen zizhiqi,50.5667,123.7167,39.87,88,100,2.37,CN,1712878124
3,3,puerto natales,-51.7236,-72.4875,37.94,91,36,5.46,CL,1712878124
4,4,nabire,-3.3667,135.4833,75.36,96,100,2.01,ID,1712878124


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [62]:
%%capture --no-display

# Configure the map plot
all_city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat", 
    geo = True,
    titles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    coastline=True,
    hover_cols='all')

# Display the map
all_city_plot

:Overlay
   .Points.I    :Points   [Lng,Lat]   (Humidity,index,City_ID,City,Max Temp,Cloudiness,Wind Speed,Country,Date)
   .Coastline.I :Feature   [Longitude,Latitude]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [63]:
# Narrow down cities that fit criteria and drop any results with null values
# The criteria is somewhere in the tropics with humidity over 75 and a max temp over 75
tropic_of_cancer = 23.44
tropic_of_capricorn = -23.44
ideal_spots_df = city_data_df.loc[(((city_data_df["Lat"] <= tropic_of_cancer) & (city_data_df["Lat"] >= 0)) | ((city_data_df["Lat"] < 0) & (city_data_df["Lat"] >= tropic_of_capricorn))) & (city_data_df["Humidity"] >= 20) & (city_data_df["Max Temp"] > 75)]

# Drop any rows with null values
ideal_spots_df.dropna(inplace=True)

# Display sample data
ideal_spots_df

/var/folders/ls/h409pvsn4l1fr1fl63lp75sm0000gn/T/ipykernel_8741/1077927420.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_spots_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bathsheba,13.2167,-59.5167,83.01,74,20,17.27,BB,1712878124
4,4,nabire,-3.3667,135.4833,75.36,96,100,2.01,ID,1712878124
10,10,cabo san lucas,22.8909,-109.9124,81.61,43,40,11.50,MX,1712878108
14,14,holualoa,19.6228,-155.9522,77.25,76,100,5.75,US,1712877898
15,15,touros,-5.1989,-35.4608,80.24,83,99,6.96,BR,1712878126
...,...,...,...,...,...,...,...,...,...,...
566,566,jutai,-5.1833,-68.9000,75.02,95,97,1.72,BR,1712878207
568,568,tual,-5.6667,132.7500,83.48,71,88,6.11,ID,1712878208
575,575,harper,4.3750,-7.7169,79.72,85,100,5.48,LR,1712878209
576,576,chauk,20.8833,94.8167,84.20,31,0,1.41,MM,1712878209


### Step 3: Create a new DataFrame called `hotel_df`.

In [64]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_spots_df[
    [
        "City",
        "Country",
        "Lat",
        "Lng",
        "Humidity"
    ]
]
    

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

/var/folders/ls/h409pvsn4l1fr1fl63lp75sm0000gn/T/ipykernel_8741/1092222906.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,bathsheba,BB,13.2167,-59.5167,74,
4,nabire,ID,-3.3667,135.4833,96,
10,cabo san lucas,MX,22.8909,-109.9124,43,
14,holualoa,US,19.6228,-155.9522,76,
15,touros,BR,-5.1989,-35.4608,83,
...,...,...,...,...,...,...
566,jutai,BR,-5.1833,-68.9000,95,
568,tual,ID,-5.6667,132.7500,71,
575,harper,LR,4.3750,-7.7169,85,
576,chauk,MM,20.8833,94.8167,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [65]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
category = f"accommodation.hotel"
params = {
    "categories":category,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
 
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
   
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
salaga - nearest hotel: No hotel found
zaraza - nearest hotel: No hotel found
mejit - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
enewetak - nearest hotel: No hotel found
sao raimundo das mangabeiras - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
arawa - nearest hotel: No hotel found
pambujan - nearest hotel: No hotel found
ta`u - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
guane - nearest hotel: No hotel found
presidente medici - nearest hotel: No hotel found
ponce - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
acarau - nearest hotel: No hotel found
sinabang - nearest hotel: No hotel found
bandarbeyla - nearest hotel: No hotel found
alenquer - nearest hotel: No hotel found
pizarro - nearest hotel: No hotel found
kinablangan - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bathsheba,BB,13.2167,-59.5167,74,Atlantis Hotel
4,nabire,ID,-3.3667,135.4833,96,Hotel Anggrek
10,cabo san lucas,MX,22.8909,-109.9124,43,Comfort Rooms
14,holualoa,US,19.6228,-155.9522,76,Kona Hotel
15,touros,BR,-5.1989,-35.4608,83,Pousada Atlântico
...,...,...,...,...,...,...
566,jutai,BR,-5.1833,-68.9000,95,No hotel found
568,tual,ID,-5.6667,132.7500,71,Suita
575,harper,LR,4.3750,-7.7169,85,No hotel found
576,chauk,MM,20.8833,94.8167,31,Royal Chauk


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [67]:
%%capture --no-display

# Configure the map plot

hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat", 
    geo = True,
    titles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    coastline=True,
    hover_cols='all')
# Display the map
hotel_plot

:Overlay
   .Points.I    :Points   [Lng,Lat]   (Humidity,index,City,Country,Hotel Name)
   .Coastline.I :Feature   [Longitude,Latitude]